In [17]:
import dotenv
import os
dotenv.load_dotenv()
from openai import AsyncOpenAI

client = AsyncOpenAI()
model = "gpt-4-1106-preview"

In [18]:
firebase_db_url = "https://therapy-assist-default-rtdb.firebaseio.com/"

async def profile_eval(messages, client) -> str:
  
  content = """
  You will be given the transcript of what a parient told a therapist during a therapy session. 
  Generate a json with the following format:
  {
  "communication_style": "one word to describe their communication style",
  "learning_style": "one word to describe their learning style",
  "interests": ["list", "of", "interests"],
  }
  """

  messages = [
      {"role": "system", "content": content},
      {'role': 'user', 'content': messages}
  ]
  
  
  response = await client.chat.completions.create(
        model=model,
        response_format={ "type": "json_object" },
        messages=messages,
        temperature=0
      )
  
  return response.choices[0].message.content
  
 
async def next_steps_eval(transcript, client) -> str:
    
  content = """
    Based on the following transcript of a therapy session, provide suggestions for what the therapist should do next to assist the patient. Consider the patient's needs, communication style, and any issues or concerns raised during the session. Provide practical and specific steps that the therapist can take in the next session or as follow-up actions.
    Generate a json with the following format:
    ["Review patient's progress", "Discuss new stressors", "Introduce new techniques"]
    """

  messages = [
        {"role": "system", "content": content},
        {'role': 'user', 'content': transcript}
    ]
    
  response = await client.chat.completions.create(
          model=model,
          response_format={ "type": "json_object" },
          messages=messages,
          temperature=0
        )
    
  next_steps = response.choices[0].message.content
  return next_steps

async def emotions_eval(transcript, client) -> str:
    
  content = """
    Analyze the following transcript of a conversation and evaluate the prevalence of four emotions: Happy, Sad, Fear, and Anger. Provide a score for each emotion as a percentage, with 0% meaning the emotion is not present at all and 100% meaning the emotion is extremely present. The scores should add up to no more than 100%.
    Format your response as a JSON object like this:
    {
      "Happy": 25,
      "Sad": 50,
      "Fear": 15,
      "Anger": 10
    }
    """

  messages = [
        {"role": "system", "content": content},
        {'role': 'user', 'content': transcript}
    ]
    
  response = await client.chat.completions.create(
          model=model,
          response_format={ "type": "json_object" },
          messages=messages,
          temperature=0
        )
    
  emotion_scores = response.choices[0].message.content
  return emotion_scores

async def therapist_eval(transcript, client):
    
  content = """
    Analyze the following transcript of a therapy session and evaluate the therapist's performance in three key areas: Language, Empathy, and Sensitivity. Provide a score for each area as a percentage, with 0% meaning very poor performance and 100% meaning excellent performance. The scores should reflect the following metrics:
    - Language: The clarity, appropriateness, and effectiveness of the therapist's communication.
    - Empathy: The therapist's ability to understand and share the feelings of the patient.
    - Sensitivity: The therapist's awareness and responsiveness to the patient's needs and emotions.
    Format your response as a JSON object like this:
    {
      "Language": 80,
      "Empathy": 75,
      "Sensitivity": 90
    }
    """

  messages = [
        {"role": "system", "content": content},
        {'role': 'user', 'content': transcript}
    ]
    
  response = await client.chat.completions.create(
          model=model,
          response_format={ "type": "json_object" },
          messages=messages,
          temperature=0
        )
    
  performance_scores = response.choices[0].message.content
  return performance_scores


def process_conversation_client_only(excerpts):
  messages = ""

  for excerpt in excerpts:
    if excerpt['role'] == "patient":
      messages += excerpt['content'] + "\n"
      
  return messages
      
def process_conversation(excerpts):
  messages = ""
  
  for excerpt in excerpts:
    messages += excerpt['role'] + ": " + excerpt['content'] + "\n"
    
  return messages

In [19]:
mock = [
  {"role": "therapist", "content": "Good morning, how are you feeling today?"},
  {"role": "patient", "content": "Honestly, I've been feeling a bit overwhelmed lately."},
  {"role": "therapist", "content": "I'm sorry to hear that. Can you tell me a bit more about what's been overwhelming you?"},
  {"role": "patient", "content": "Work has been extremely stressful, and it's been hard to balance that with family life."},
  {"role": "therapist", "content": "Balancing work and family can certainly be challenging. How have you been coping with this stress?"},
  {"role": "patient", "content": "I try to relax in the evenings, but I'm finding it hard to switch off from work."},
  {"role": "therapist", "content": "It's important to have that separation. Have you tried any relaxation techniques or activities that might help?"},
  {"role": "patient", "content": "I've tried a bit of meditation but haven't stuck with it consistently."},
  {"role": "therapist", "content": "Meditation can be a great tool. Consistency is key, though. Maybe we can explore some strategies to help make it a regular part of your routine."},
  {"role": "patient", "content": "That sounds like a good idea. I'm willing to try anything that might help."},
  {"role": "therapist", "content": "Great, let's work on that together. It's also important to address the root causes of your stress. Let's explore what changes you can make at work to reduce your stress levels."}
]


In [20]:
mock_patient = process_conversation_client_only(mock)
mock_transcript = process_conversation(mock)
print(mock_patient)
print(mock_transcript)

Honestly, I've been feeling a bit overwhelmed lately.
Work has been extremely stressful, and it's been hard to balance that with family life.
I try to relax in the evenings, but I'm finding it hard to switch off from work.
I've tried a bit of meditation but haven't stuck with it consistently.
That sounds like a good idea. I'm willing to try anything that might help.

therapist: Good morning, how are you feeling today?
patient: Honestly, I've been feeling a bit overwhelmed lately.
therapist: I'm sorry to hear that. Can you tell me a bit more about what's been overwhelming you?
patient: Work has been extremely stressful, and it's been hard to balance that with family life.
therapist: Balancing work and family can certainly be challenging. How have you been coping with this stress?
patient: I try to relax in the evenings, but I'm finding it hard to switch off from work.
therapist: It's important to have that separation. Have you tried any relaxation techniques or activities that might hel

In [21]:
eval_profile = await profile_eval(mock_patient, client)
print(eval_profile)
eval_next_steps = await next_steps_eval(mock_transcript, client)
print(eval_next_steps)
eval_emotions = await emotions_eval(mock_transcript, client)
print(eval_emotions)
eval_therapist = await therapist_eval(mock_transcript, client)
print(eval_therapist)


{
  "communication_style": "open",
  "learning_style": "auditory",
  "interests": ["work", "family", "relaxation", "meditation"]
}
{
  "suggestions": [
    "Review patient's progress with previous coping strategies",
    "Discuss new stressors at work and in family life",
    "Explore and introduce new relaxation techniques",
    "Develop a consistent meditation practice",
    "Identify changes that can be made at work to reduce stress",
    "Create a work-life balance plan",
    "Set goals for stress management",
    "Follow up on the implementation of relaxation techniques in the next session"
  ]
}
{
  "Happy": 0,
  "Sad": 40,
  "Fear": 30,
  "Anger": 0
}
{
  "Language": 90,
  "Empathy": 85,
  "Sensitivity": 88
}
